In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"for dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))"

In [2]:
df=pd.read_csv('/kaggle/input/extracting-attributes-from-fashion-images/train.csv')

In [3]:
df.head()

,file_name,label
0,Image_train_00001.jpg,0
1,Image_train_00002.jpg,1
2,Image_train_00003.jpg,0
3,Image_train_00004.jpg,0
4,Image_train_00005.jpg,0


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [5]:
data_gen=ImageDataGenerator(zoom_range=0.20,shear_range=0.20,rescale=1/255,horizontal_flip=True)

In [6]:
df['label'] = df['label'].astype(str)

In [7]:
train_data=data_gen.flow_from_dataframe(df,'/kaggle/input/extracting-attributes-from-fashion-images/train',
                             x_col='file_name',y_col='label',class_mode='categorical')

Found 18391 validated image filenames belonging to 7 classes.


In [8]:
from tensorflow.keras.applications.vgg16 import VGG16

In [9]:
model_vgg=VGG16(include_top=False,input_shape=(256,256,3))

58889256/58889256 [==============================] - 0s 0us/step


In [10]:
model_vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [11]:
for layer in model_vgg.layers:
    layer.trainable=False

In [12]:
model_vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [13]:
from tensorflow.keras.layers import Flatten,Dense
from tensorflow.keras.models import Model

In [14]:
last_layer=Dense(7,activation='softmax')(Flatten()(model_vgg.output))

In [15]:
model=Model(inputs=model_vgg.input,outputs=last_layer)

model.compile(loss='categorical_crossentropy',metrics=['accuracy'])

In [16]:
model=Model(inputs=model_vgg.input,outputs=last_layer)

model.compile(loss='categorical_crossentropy',metrics=['accuracy'])

In [17]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0   

In [18]:
train_data.class_indices

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6}

In [19]:
model.fit(train_data,epochs=20)

Epoch 1/20
575/575 [==============================] - 399s 678ms/step - loss: 2.6315 - accuracy: 0.4162
Epoch 2/20
575/575 [==============================] - 300s 522ms/step - loss: 2.1062 - accuracy: 0.5086
Epoch 3/20
575/575 [==============================] - 298s 519ms/step - loss: 1.9609 - accuracy: 0.5415
Epoch 4/20
575/575 [==============================] - 298s 518ms/step - loss: 1.8375 - accuracy: 0.5717
Epoch 5/20
575/575 [==============================] - 300s 522ms/step - loss: 1.7791 - accuracy: 0.5840
Epoch 6/20
575/575 [==============================] - 298s 517ms/step - loss: 1.7208 - accuracy: 0.5972
Epoch 7/20
575/575 [==============================] - 300s 521ms/step - loss: 1.6786 - accuracy: 0.6100
Epoch 8/20
575/575 [==============================] - 300s 522ms/step - loss: 1.6296 - accuracy: 0.6192
Epoch 9/20
575/575 [==============================] - 303s 526ms/step - loss: 1.5860 - accuracy: 0.6300
Epoch 10/20
575/575 [==============================] - 303s 527m

In [20]:
model.save('/kaggle/working/fashion_images.h5')

In [21]:
import pandas as pd
import cv2,os

In [22]:
filenames=os.listdir('/kaggle/input/extracting-attributes-from-fashion-images/test/')

In [23]:
for file in filenames[0:6]:
    print(file)

Image_test_03058.jpg
Image_test_02479.jpg
Image_test_05379.jpg
Image_test_01202.jpg
Image_test_03064.jpg
Image_test_02787.jpg


In [24]:
def predict(path):
    img=cv2.imread(path)
    img=cv2.resize(img,(256,256))
    img=img.reshape(1,256,256,3)/255
    i=model.predict_on_batch(img).argmax()
    return i

In [27]:
for file in filenames:
    predict('/kaggle/input/extracting-attributes-from-fashion-images/test/Image_test_00001.jpg')

3

In [ ]:
# ResNet Model

In [28]:
from tensorflow.keras.applications.resnet_v2 import ResNet152V2

In [29]:
model_Resnet=ResNet152V2(include_top=False,input_shape=(256,256,3))

234545216/234545216 [==============================] - 1s 0us/step


In [30]:
model_Resnet.summary()

Model: "resnet152v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                        

In [31]:
for layer in model_Resnet.layers:
    layer.trainable=False

In [32]:
model_Resnet.summary()

Model: "resnet152v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                        

In [33]:
from tensorflow.keras.layers import Flatten,Dense
from tensorflow.keras.models import Model

In [34]:
last_layer=Dense(7,activation='softmax')(Flatten()(model_Resnet.output))

In [35]:
model_RES=Model(inputs=model_Resnet.input,outputs=last_layer)

model_RES.compile(loss='categorical_crossentropy',metrics=['accuracy'])

In [36]:
model_RES.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [37]:
train_data.class_indices

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6}

In [38]:
model_RES.fit(train_data,epochs=30)

Epoch 1/30
575/575 [==============================] - 331s 558ms/step - loss: 13.4794 - accuracy: 0.4857
Epoch 2/30
575/575 [==============================] - 316s 549ms/step - loss: 11.0720 - accuracy: 0.5817
Epoch 3/30
575/575 [==============================] - 315s 548ms/step - loss: 9.9856 - accuracy: 0.6274
Epoch 4/30
575/575 [==============================] - 321s 558ms/step - loss: 9.3720 - accuracy: 0.6539
Epoch 5/30
575/575 [==============================] - 313s 544ms/step - loss: 8.8346 - accuracy: 0.6692
Epoch 6/30
575/575 [==============================] - 314s 545ms/step - loss: 8.3009 - accuracy: 0.6933
Epoch 7/30
575/575 [==============================] - 315s 548ms/step - loss: 7.9171 - accuracy: 0.7070
Epoch 8/30
575/575 [==============================] - 329s 572ms/step - loss: 7.6896 - accuracy: 0.7181
Epoch 9/30
575/575 [==============================] - 314s 545ms/step - loss: 7.3748 - accuracy: 0.7300
Epoch 10/30
575/575 [==============================] - 315s 54

In [ ]:
model_RES.save('/kaggle/working/fasion_images_resnet.h5')